In [24]:
import os     
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import Activation,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
tf.__version__

'2.7.0'

In [25]:
classes = ['black', 'brown', 'panda', 'polar']

In [26]:
idg=ImageDataGenerator(rescale=1./255,dtype = ('float32'),
                                   rotation_range=40,        #隨機旋轉0~40度
                                   width_shift_range=0.4,    #隨機水平偏移的幅度  參數0~1之間  隨機偏移距離0~圖片長*參數
                                   height_shift_range=0.4,   #隨機垂直偏移的幅度  參數0~1之間  隨機偏移距離0~圖片寬*參數
                                   shear_range=0.4,         #按比例平移
                                    fill_mode='nearest',
                                   validation_split=0.2,
                                   zoom_range=0.4,          #隨機縮放的幅度 參數0~1放大   大於1變小
                                   horizontal_flip=True)
idg2=ImageDataGenerator(rescale=1./255)

In [27]:
train_directory=idg.flow_from_directory(
    directory=os.path.join("./train"),
    target_size=(224, 224),
    classes=classes,
    batch_size=32,
    class_mode='categorical',
    subset='training')

Found 120 images belonging to 4 classes.


In [28]:
validation_directory=idg.flow_from_directory(
    directory=os.path.join("./train"),
    target_size=(224, 224),
    classes=classes,
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 27 images belonging to 4 classes.


In [29]:
model = Sequential()

model.add(Conv2D(filters=16,
                 kernel_size=(3,3),
                 padding='same',
                 input_shape=(224,224,3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=16,
                 kernel_size=(3,3),
                 padding='same',
                 input_shape=(224,224,3), 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2,activation='softmax'))

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 112, 112, 16)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 112, 112, 16)      2320      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 56, 56, 16)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 50176)             0         
                                                                 
 dropout_4 (Dropout)         (None, 50176)            

In [30]:
adamm=Adam(lr=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=adamm,metrics=['accuracy']) 

/usr/local/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [31]:
stop=EarlyStopping(monitor='val_loss', patience=5,verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    './model/',
    monitor='val_loss',
    save_best_only=True,
    verbose=0 ) 

In [32]:
fit_history=model.fit_generator(train_directory,
                                steps_per_epoch=100,
                                epochs=20,
                                validation_data=validation_directory,
                                validation_steps=20,
                                callbacks=[stop,checkpoint]) 

/var/folders/hk/4rr3lz8x5vxgx7z4sz_tcqzc0000gn/T/ipykernel_8288/59610334.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history=model.fit_generator(train_directory,


ImportError: Image transformations require SciPy. Install SciPy.

In [ ]:
model = load_model('./model/')

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_history,train_acc,test_acc):
    plt.plot(train_history.history[train_acc])
    plt.plot(train_history.history[test_acc])
    plt.title('Train History')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
show_train_history(fit_history,'accuracy','val_accuracy')

In [ ]:
show_train_history(fit_history,'loss','val_loss')

In [ ]:
test_path = './test/test.csv'
df = pd.read_csv(test_path, sep=',', skiprows=1,  names=['id', 'label'])
df['id'] = df['id'].astype('str')

test_datagen = ImageDataGenerator(rescale=1/255.)

test_directory = test_datagen.flow_from_dataframe(df, './test/testtest',
                                                x_col='id', y_col='label',
                                                target_size=(224,224),
                                                interpolation='bicubic',
                                                class_mode=None,
                                                shuffle=False,
                                                batch_size=1)

In [ ]:
predict = model.predict_generator(test_directory,verbose=1)
predict[:10]